*This notebook gets the raw samknows traceroute measurement data from `curr_traceroute` table; prunes the data to fetch (probeid, timestamp, last_hop_latencies) then inserts entire dataframe in `sk_latencies_last_hop_agg` table.*
- - - 

In [1]:
import sqlite3
import pandas as pd
import requests
import ipaddress
import time

In [2]:
DB_LOCATION = 'lastmile.db'
CURR_TRACEROUTE = 'curr_traceroute'
SK_LATENCIES_AGG = 'sk_latencies_last_hop_agg'

In [3]:
con = sqlite3.connect(DB_LOCATION)

In [4]:
query = '''SELECT    unit_id as probeid
                   , dtime as timestamp
                   , GROUP_CONCAT(hop, ';') as hops
                   , GROUP_CONCAT(rtt_avg, ';') as rtts
                   , '' as last_hop
                   , '' as last_hop_avg
           FROM      %s
           GROUP BY  probeid, timestamp
        '''%(CURR_TRACEROUTE)

df = pd.read_sql(query, con)

In [5]:
df.head()

,probeid,timestamp,hops,rtts,last_hop,last_hop_avg
0,14190,2014-07-28 05:42:57,2;3;4;5;6;7;8;9;10;11,1211;1829;1874;7781;8786;17540;12890;7868;7757...,,
1,14190,2014-07-28 05:43:01,1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16,377;1280;1956;4216;11309;10287;16797;9089;1151...,,
2,14190,2014-07-28 13:41:21,1;2;3;4;5;6;7;8;9;10;11,424;1383;1827;2183;8265;12836;19957;16784;1227...,,
3,14190,2014-07-28 13:41:24,1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16,362;1669;1723;71457;10363;10321;10385;9262;942...,,
4,14190,2014-07-28 17:43:23,1;2;3;4;5;6;7;8;9;10;11,395;1297;1807;2262;11179;10952;13786;7872;1209...,,


In [6]:
def get_last_hop_rtt(rtts): 
    try: last_hop_rtt = rtts.split(';')[-1]; return float(last_hop_rtt)/1000
    except Exception as e: print('*', end=''); return None
    
def get_last_hop(hops): 
    try: last_hop = hops.split(';')[-1]; return float(last_hop)
    except Exception as e: print('*', end=''); return None

df['last_hop_avg'] = df['rtts'].apply(get_last_hop_rtt)
df['last_hop']     = df['hops'].apply(get_last_hop)

In [7]:
del df['hops']
del df['rtts']

In [8]:
df.head()

,probeid,timestamp,last_hop,last_hop_avg
0,14190,2014-07-28 05:42:57,11,7.668
1,14190,2014-07-28 05:43:01,16,77.968
2,14190,2014-07-28 13:41:21,11,7.719
3,14190,2014-07-28 13:41:24,16,78.150
4,14190,2014-07-28 17:43:23,11,7.639


In [9]:
df.count()

probeid         789124
timestamp       789124
last_hop        789124
last_hop_avg    789124
dtype: int64

In [10]:
cur = con.execute('pragma foreign_keys=ON')

In [11]:
index_label = ['probeid', 'timestamp']
df = df.set_index(index_label)

In [12]:
df.head()

last_hop  last_hop_avg
probeid timestamp                                  
14190   2014-07-28 05:42:57        11         7.668
        2014-07-28 05:43:01        16        77.968
        2014-07-28 13:41:21        11         7.719
        2014-07-28 13:41:24        16        78.150
        2014-07-28 17:43:23        11         7.639

In [13]:
df.to_sql(  '%s'%SK_LATENCIES_AGG
          , con
          , flavor='sqlite'
          , if_exists = 'append'
          , index_label = index_label
         )

In [14]:
con.commit()
con.close()